# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---

# <h1><center>Clase 19: Aprendizaje Supervisado: Regresiones</center></h1>

En este notebook aplicaremos las herramientas implementadas en las librerías de Python `statsmodels` y `scipy` para realizar regresiones.

Para implementar regresiones lineales y predecir los outcomes para distintas variables de entrada, podemos usar distintas librerías de `python`:

* [scipy.stats](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html).
* [statsmodels](http://www.statsmodels.org/stable/regression.html) y 
* [scikit-learn (sklearn)](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).


Para efectos de regresiones lineales simples,`scipy`, `statsmodels` y `sklearn` hacen lo mismo. Más en general,  `statsmodels` tiende a ser más fácil para problemas de inferencia (econtrar los parámetros de la regresión y analizar sus incertezas) mientras que  `sklearn` tiene un enfoque de machine learning y es más práctico para realizar predicciones.

## Caso de Estudio: rendimiento de algunos autos


### Descripción del Dataset

Para este ejercicio, tenemos como ejemplo un dataset con datos de automóviles, extraidos del repositorio UCI Machine Learning Repository:

https://archive.ics.uci.edu/ml/datasets/auto+mpg


*The data concerns city-cycle fuel consumption in miles per gallon, to be predicted in terms of 3 multivalued discrete and 5 continuous attributes." (Quinlan, 1993)* <br>

Los datos incluyen datos sobre características del diseño e ingeniería de distintos modelos de autos, como se describe a continuación:

- `mpg`: rendimiento de combustible en millas/galón. 
- `cylinders`: número de cilindros. is Number of cylinders, 
- `displacement`: desplazamiento (cu.in.), 
- `horsepower`: potencia en "caballos de fuerza". 
- `weight`: peso (en miles de libras).
- `acceleration`: tiempo en recorrer 1/4 de milla desde el reposo.
- `origin`: origen 
- `car name`: nombre del modelo 

### Objetivo

El objetivo es predecir el consumo de un vehículo, en función de sus parámetros de diseño e ingeniería.

## 1. Datos a analizar

El archivo `auto-mpg.csv` será la base que usaremos para este trabajo.

In [ ]:
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
#%matplotlib inline

## 2. EDA

### 2.1 Exploración inicial

Exploremos primero un poco los datos para entender qué relaciones podrían existir.

In [ ]:
dfautos = pd.read_csv("data//auto-mpg.csv")
dfautos.head()

In [ ]:
dfautos.dropna(axis=0, inplace=True)
dfautos.info()

In [ ]:
dfautos['horsepower'].unique()

In [ ]:
dfautos = dfautos[dfautos['horsepower']!='?']

In [ ]:
dfautos['origin'] = dfautos['origin'].astype('category')
dfautos['car name'] = dfautos['car name'].astype('category')
dfautos['cylinders'] = dfautos['cylinders'].astype('category')
dfautos['horsepower'] = dfautos['horsepower'].astype('float')

In [ ]:
# explorar datos
sns.pairplot(dfautos, y_vars='mpg')

In [ ]:
sns.heatmap(dfautos.corr(numeric_only=True), annot=True)

### 2.2 Rendimiento según diferentes dimensiones

Exploremos cuál es el rendimiento promedio para distintas combinaciones de cilindros y hp.

In [ ]:
sns.pairplot(data=dfautos, y_vars='mpg', x_vars='horsepower', hue='cylinders', height=8)

In [ ]:
sns.lmplot(data=dfautos, y='mpg', x='horsepower', hue='cylinders', height=8)

## 3. Regresión Lineal

### 3.1 Armando el modelo

- Nota sobre constantes:

Supongamos un caso sencillo, en que tenemos 2 observaciones con un predictor y una variable de respuesta cada una. Tendríamos por lo tanto el siguiente sistema de ecuaciones para el modelo de regresión lineal simple:
 $$y_1=\beta_0 + \beta_1\cdot x_1$$ $$y_2=\beta_0 + \beta_1\cdot x_2$$ <BR>
    
En notación matricial, esto sería: 
    
$$
\left [
\begin{array}{c}
y_1 \\ y_2 \\
\end{array}
\right] =
\left [
\begin{array}{cc}
1& x_1 \\ 1 & x_2 \\
\end{array}
\right] 
\cdot
\left [
\begin{array}{c}
\beta_0 \\ \beta_1 \\
\end{array}
\right]
$$
<BR><BR>
    
`sklearn` agrega automáticamente la constante `1`, mientras que en  `statsmodels` hay que agregarla explícitamente usando `sm.add_constant`

In [ ]:
import statsmodels.api as sm

x = np.array(dfautos.horsepower)
y = np.array(dfautos.mpg)

In [ ]:
X = sm.add_constant(x)
X

In [ ]:
# modelo OLS (ordinary least squares) entrenado con los datos
regr_sm = sm.OLS(y, X)

# aplicar fit para entrenar y guardar el modelo
results_sm = regr_sm.fit()

# entregar informacion general del modelo
results_sm.summary()

In [ ]:
# cpeficientes beta
beta0 = results_sm.params[0]
beta1 = results_sm.params[1]
beta0, beta1

### 3.2 Visualizando datos y predicciones

Ahora queremos ver los datos junto a las predicciones hachas por el modelo

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6))

grid = np.linspace(np.min(dfautos.horsepower), np.max(dfautos.horsepower), 100)

y_fit = beta0 + beta1*grid

ax.plot(grid, y_fit, '--', color='g', label='linear fit')
ax.plot(dfautos.horsepower, dfautos.mpg, 'o', color='g', alpha=0.5, label='observed') 
ax.set_xlabel("HP", fontsize=12)
ax.set_ylabel("MPG", fontsize=12)
ax.legend();

Veamos ahora para el peso en `weight`.

In [ ]:
# preparar datos
x = np.array(dfautos.weight)
y = np.array(dfautos.mpg)
X = sm.add_constant(x)

# realizar modelo
regr_sm = sm.OLS(y, X)
results_sm = regr_sm.fit()

# extraer par'ametros
beta0 = results_sm.params[0]
beta1 = results_sm.params[1]

# visualizar
fig, ax = plt.subplots(1,1, figsize=(8,6))
grid = np.linspace(np.min(dfautos.weight), np.max(dfautos.weight), 100)
y_fit_lineal = beta0 + beta1*grid

ax.plot(grid, y_fit_lineal, '--', color='b', label='Linear fit')
ax.plot(dfautos.weight, dfautos.mpg, 'o', color='b', alpha=0.5, label='Observed') 

ax.set_xlabel("Weight", fontsize=12)
ax.set_ylabel("MPG", fontsize=12)
ax.legend()

In [ ]:
results_sm.rsquared

### 3.3 Regresión  Polinomial

En una regresión polinomial, usamos un **modelo lineal** para estimar una **función no lineal** (i.e., una función con términos polinomiales). Por ejemplo, 

$y = \beta_0 + \beta_1x_i + \beta_2x_i^{2}$

Es un modelo _lineal_ porque resolvemos una ecuación lineal para obtener los coeficientes $\beta$.

In [ ]:
import statsmodels.formula.api as smf

#¿Es un buen ajuste? tal vez podemos probar con una funcion no lineal
dfautos['wt2'] = dfautos['weight']**2

# Calcular regresión lineal con 
lm1 = smf.ols('mpg ~ weight + wt2', data=dfautos).fit()

# Print the estimated parameters
print(lm1.params, lm1.rsquared)
lm1.summary()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6))

xgrid = np.linspace(np.min(dfautos.weight), np.max(dfautos.weight), 100)

y_fit_poly = lm1.params[0] + lm1.params[1]*xgrid + lm1.params[2]*xgrid**2

ax.plot(xgrid, y_fit_poly, '--', color='r', label='Polynomial fit', lw=2)
ax.plot(grid, y_fit_lineal, '-', color='green', label='Linear fit')

ax.plot(dfautos.weight, dfautos.mpg, 'o', color='b', alpha=0.3, label='Observed') 

ax.set_xlabel("Weight", fontsize=12)
ax.set_ylabel("MPG", fontsize=12)
ax.legend()

In [ ]:
from sklearn.metrics import mean_squared_error

y_predict_lin = beta0 + beta1*x
print('MSE lineal:', mean_squared_error(y, y_predict_lin))


y_predict_poly = lm1.params[0] + lm1.params[1]*x + lm1.params[2]*x**2
print('MSE poly:', mean_squared_error(y, y_predict_poly))

## 4. Regresión Multilineal

### 4.1 Armando el modelo

Supongamos ahora un modelo incluyendo múltiples variables:

$$\text{mpg} \approx \beta_0 + \beta_1\text{disp} + \beta_2\text{hp} + \beta_3\text{wt} +\beta_4\text{wt}^2 $$

In [ ]:
import statsmodels.formula.api as smf

def build_xmatrix(df, columns, cols_squared=[]):
    x_matrix = df[columns].copy()
    
    for col in cols_squared:
        x_matrix['%s_2'%col] = df[col]**2
    
    x_matrix = sm.add_constant(x_matrix)
    return(x_matrix)

In [ ]:
columns = ['displacement','horsepower','weight']
cols_squared = ['weight']

x_matrix = build_xmatrix(dfautos, columns, cols_squared)
x_matrix.head()

In [ ]:
import statsmodels.api as sm
from statsmodels.api import OLS

y = dfautos.mpg
fitted_model = OLS(y, x_matrix, hasconst=True).fit()
fitted_model.summary()

### 4.3 Visualizando el modelo

Ahora veamos cómo visualizar los resultados de nuestro modelo.

In [ ]:
fig = plt.figure(figsize=(12,4))

x_wt = np.arange(dfautos['weight'].min(), dfautos['weight'].max(), 0.01)
wtcoef = fitted_model.params.weight
wt2coef = fitted_model.params.weight_2
y_wt = wtcoef*x_wt + wt2coef*x_wt**2

ax1 = fig.add_subplot(131)
ax1.plot(x_wt, y_wt)
ax1.set_title("Efecto del peso del auto ('weight', 'weight'^2)")
ax1.set_xlabel("Weight")
ax1.set_ylabel("Contribución al rendimiento")

hpcoef = fitted_model.params.horsepower
x_hp = np.arange(dfautos['horsepower'].min(), dfautos['horsepower'].max(), 0.01)
y_hp = hpcoef*x_hp

ax2 = fig.add_subplot(132)
ax2.plot(x_hp, y_hp)
ax2.set_title("Efecto de HP")
ax2.set_xlabel("HP")
ax2.set_ylabel("Contribución al rendimiento")

dispcoef = fitted_model.params.displacement
x_disp = np.arange(dfautos['displacement'].min(), dfautos['displacement'].max(), 0.01)
y_disp = dispcoef*x_disp

ax3=fig.add_subplot(133)
ax3.plot(x_disp, y_disp)
ax3.set_title("Efecto del Desplazamiento")
ax3.set_xlabel("Disp")
ax3.set_ylabel("Contribución al rendimiento")

plt.show()

## 5. Problemas a considerar

### 5.1 Colineares

Cuando tenemos datos que tienen colinearidad entre ellos, podemos tener problemas al aplicar regresiones.

NOTA: para correr este gráfico en forma interactiva, deben instalar la librería `ipympl`.

In [ ]:
#%% Generate Data
# for creating a responsive plot
#%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D

print('Generating Data')
# sample size
T = 200
# number of factors
m = 2
# generate factor values
x = rnd.uniform(size=(T,m))

#%% Model
# real world factors
betaR = np.array([[2], [1]])

# noise variance
s = .4
# observed data
y = np.dot(x, betaR) + rnd.normal(0, s, size=(T,1))

fig = plt.figure(1)
ax = fig.add_subplot(projection='3d')
ax.scatter(x[:,0], x[:,1], y, c='r', alpha=0.1)
ax.set_xlabel('x_1')
ax.set_ylabel('x_2')
ax.set_zlabel('y')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_zlim([-2, 5])
plt.show()

In [ ]:
#%% Regression
from sklearn import linear_model

print('Regression Model')
# make linear regression model
regr = linear_model.LinearRegression()

# fit model with data
regr.fit(x, y)
r2 = regr.score(x, y)
# get coefficients and fitness
betaReg = regr.coef_[0]
print('Coefficients: ' + str(betaReg))
print('r2 = ' + str(r2))

# build plane
Xp, Yp = np.meshgrid(np.arange(0, 1.1, .5), np.arange(0, 1.1, .5))
Zp = betaReg[0] * Xp + betaReg[1] * Yp
# plot plane
fig = plt.figure(2)
ax = fig.add_subplot(projection='3d')

ax.plot_surface(Xp, Yp, Zp, linewidth=0, antialiased=False, alpha=0.9)
ax.scatter(x[:,0], x[:,1], y, c='r', alpha=0.1)
ax.set_xlabel('x_1')
ax.set_ylabel('x_2')
ax.set_zlabel('y')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_zlim([-2, 5])
plt.show()

In [ ]:
#%% Make collinear data
print('Generating Collinear Data')
# generate factor values
xCol = rnd.uniform(size=(T,m))
xCol[:,1] = xCol[:,0] + rnd.normal(0, .0001, size=T)

# observed data
yCol = np.dot(xCol, betaR) + rnd.normal(0, s, size=(T,1))

fig = plt.figure(3)
ax = fig.add_subplot(projection='3d')

ax.scatter(xCol[:,0], xCol[:,1], yCol, c='r', alpha=0.1)
ax.set_xlabel('x_1')
ax.set_ylabel('x_2')
ax.set_zlabel('y')
ax.set_xlim([0, 1])
ax.set_ylim([0, 1])
ax.set_zlim([-2, 5])
plt.show()

In [ ]:
#%% Collinear Regression
# make linear regression model
regrCol = linear_model.LinearRegression()
# fit model with data
regrCol.fit(xCol, yCol)
r2Col = regrCol.score(xCol, yCol)
# get coefficients
betaCol = regrCol.coef_[0]
print('Coefficients: ' + str(betaCol))
print('r2 = ' + str(r2Col))

### La Paradoja de Simpson

Otro elemento clásico a considerar al realizar regresiones.

In [ ]:
#%matplotlib inline
df = pd.read_csv("data//berkeley.csv")
df.head()

In [ ]:
gender_group = df.groupby(['Gender', 'Admission']).size().unstack()
gender_group

In [ ]:
gender_group.apply(lambda x:100*x/x.sum(), axis=1).plot(kind='barh', stacked=True, legend=False)
plt.legend(['Accepted', 'Rejected'])

¿Fue el sistema de admisión de Berkeley sesgado?

In [ ]:
gender_group_major = df.groupby(['Major', 'Gender', 'Admission']).size().unstack()
gender_group_major

In [ ]:
gender_group_major.apply(lambda x:100*x/x.sum(), axis=1).plot(kind='barh', stacked=True, legend=False)
plt.legend(['Accepted', 'Rejected'])

Veamos por qué pasa esto con otro ejemplo

In [ ]:
df = pd.read_csv("data//clientes.csv")
df.head()

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
plt.plot(df['Time_spent'], df['Satisfaction'], '.')

In [ ]:
import statsmodels.api as sm

regresion = sm.OLS(df["Satisfaction"], sm.add_constant(df["Time_spent"])).fit()
print(regresion.params)

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
plt.plot(df['Time_spent'], df['Satisfaction'], '.')
plt.xlabel('Time_spent')
plt.ylabel('Satisfaction')
x = np.array([0.2, 1.6])
plt.plot(x, regresion.params.const + regresion.params.Time_spent*x, '-')

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
plt.plot(df['Time_spent'][df['Country']==0], df['Satisfaction'][df['Country']==0], 'r.')
plt.plot(df['Time_spent'][df['Country']==1], df['Satisfaction'][df['Country']==1], 'g.')
plt.plot(df['Time_spent'][df['Country']==2], df['Satisfaction'][df['Country']==2], 'b.')
plt.xlabel('Time_spent')
plt.ylabel('Satisfaction')

In [ ]:
reg0 = sm.OLS(df["Satisfaction"][df['Country']==0], sm.add_constant(df["Time_spent"][df['Country']==0])).fit()
reg1 = sm.OLS(df["Satisfaction"][df['Country']==1], sm.add_constant(df["Time_spent"][df['Country']==1])).fit()
reg2 = sm.OLS(df["Satisfaction"][df['Country']==2], sm.add_constant(df["Time_spent"][df['Country']==2])).fit()

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
plt.plot(df['Time_spent'][df['Country']==0], df['Satisfaction'][df['Country']==0], 'r.')
plt.plot(df['Time_spent'][df['Country']==1], df['Satisfaction'][df['Country']==1], 'g.')
plt.plot(df['Time_spent'][df['Country']==2], df['Satisfaction'][df['Country']==2], 'b.')

x = np.array([0.2, 1.6])
plt.plot(x, reg0.params.const + reg0.params.Time_spent*x, '-r')
plt.plot(x, reg1.params.const + reg1.params.Time_spent*x, '-g')
plt.plot(x, reg2.params.const + reg2.params.Time_spent*x, '-b')
plt.xlabel('Time_spent')
plt.ylabel('Satisfaction')

In [ ]:
import statsmodels.formula.api as smf

multi_model = smf.mixedlm("Satisfaction ~ Time_spent", data=df, groups=df["Country"], re_formula="Time_spent").fit()
print(multi_model.fe_params)
print(multi_model.random_effects)